In [1]:
# This demo implements the recursive linearization method.

from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
import numpy as np
from numpy.linalg import lstsq
from ngsolve import * 
from ngsolve.webgui import Draw
import multiprocessing
from joblib import Parallel, delayed
import numpy as np
import matplotlib.pyplot as plt
import time

dg = DataGenerator(maxh = (0.2, 0.4))

n_process = 42 # deps on how many cores on your computer

medium = MediumGenerator(cosine_func) # MediumGenerator(disk_func)

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)

params  = [{"type": 0, "x": 0.6 ,"y": 0.4, "r": 0.2, "v": 1}, 
           {"type": 0, "x": 0.7, "y": -0.6, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.8, "y": -0.3, "r": 0.4, "v": 1},
           {"type": 0, "x": -0.5, "y": 0.7, "r": 0.5, "v": 1},
           {"type": 0, "x": -0.6 ,"y": 0.2, "r": 0.6, "v": 1}, 
           {"type": 0, "x": -0.7, "y": -0.9, "r": 0.5, "v": 1},
           {"type": 0, "x": -0.1, "y": -0.3, "r": 0.4, "v": 1},
           {"type": 0, "x": -0.2, "y": 0.3, "r": 0.3, "v": 1}]

permittivity  = medium.generate(params)


Mesh generation took 0.05040406109765172 seconds


In [2]:
Draw(permittivity, dg.mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [3]:
def single_loop(i, bp, freq, inc_dir, out_dir):        
    mat = []
    vec = []
    
    kx = 2 * pi * freq * cos(inc_dir[i])
    ky = 2 * pi * freq * sin(inc_dir[i])

    psi = CF((exp(1j * kx * x) * exp(1j * ky * y)))

    u_scat = dg.solve(kx, ky, permittivity)
    
    uB_scat_psi = dg.solve(kx, ky, bp)


    for j_angle in out_dir:
        p_kx = 2 * pi * freq * cos(j_angle)
        p_ky = 2 * pi * freq * sin(j_angle)

        phi = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
        
        uB_scat_phi = dg.solve(p_kx, p_ky, bp)

        true_val = Integrate( (permittivity - bp) * (uB_scat_phi + phi) *  (u_scat + psi) * (IfPos((x)**2 + (y)**2 - (1.5) **2, 0, 1)) , dg.mesh)

        test_func = dg.fes.TestFunction()

        linear_form = LinearForm(dg.fes)

        linear_form += test_func * (uB_scat_phi + phi) *  (uB_scat_psi + psi) * (IfPos((x)**2 + (y)**2 - (1.5) **2,
                    0,
                    1))  * dx 

        linear_form.Assemble()
            
        mat.append(linear_form.vec.FV().NumPy())
        vec.append(true_val)

    return mat, vec

def assemble_linear_sys(bp, freq, n_in_dir=16, n_out_dir=16):
    A = Matrix(2 * n_in_dir * n_out_dir, dg.fes.ndof, complex=True)
    b = Vector(2 * n_in_dir * n_out_dir, complex=True)

    # incident and test directions are aligned to maximize the captured frequency domain
    inc_dir = np.linspace(0, 2 * np.pi, n_in_dir, endpoint=False)
    out_dir = np.linspace(0, 2 * np.pi, n_out_dir, endpoint=False)

    start = time.time()
    
    with multiprocessing.Pool(n_process) as p:
        tasks = [p.apply_async(single_loop, [i, bp, freq, inc_dir, out_dir]) for i in range(n_in_dir)]

        finished = {}
        
        # postprocessing step
        while len(finished) != len(tasks):
            for i, task in enumerate(tasks):
                if task.ready():
                    finished[i] = task.get()
                    for j in range(n_out_dir):
                        index = i * n_out_dir + j
                        A.NumPy()[2 * index, :]     = finished[i][0][j].real
                        A.NumPy()[2 * index + 1, :] = finished[i][0][j].imag
                        b[2 * index]        = finished[i][1][j].real
                        b[2 * index + 1]    = finished[i][1][j].imag
                    
    end_time = time.time()

    print('freq: {}, parallel time: {}'.format(freq, end_time - start))
    
    return A,  b



In [4]:
freq = 0.0

for iter in range(20):
    start_time = time.time()
    freq += 0.1
    err = Integrate((background_permittivity-permittivity)*Conj(background_permittivity-permittivity), dg.mesh).real
    
    A, b = assemble_linear_sys(background_permittivity, freq, n_in_dir=16, n_out_dir=4)
    
    v = lstsq(A.NumPy(),  b.NumPy(), rcond=1e-2)[0]
    
    permittivity_update =  GridFunction(dg.fes)
    permittivity_update.vec.data = v.real
    background_permittivity = background_permittivity + permittivity_update
    
    end_time = time.time()
    
    scene = Draw(background_permittivity, dg.mesh)
    clear_output() # redraw the scene with a new height
    scene.Draw(height="50vh")
    
    print('iter: {:4d}, freq: {:6.2f}, error squared: {:6.4e}, time: {:6.2f}'.format(iter, freq, err, end_time - start_time))

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

iter:   19, freq:   2.00, error squared: 5.1324e-02, time:  14.22
